In [1]:
import socket
import threading
import json
import time

In [2]:
connections = {}
server_sockets = []

In [3]:
def handle_client(conn, addr, client_name):
    """
    Handles communication with a connected client.
    """
    print(f"{client_name} connected from {addr}")
    connections[client_name] = conn
    try:
        while True:
            data = conn.recv(1024)  # Receive data
            if not data:
                break

            message = data[2:].decode('utf-8')

            message_dict = json.loads(message)

            print(f"Received from {client_name}: {message_dict}")
    except ConnectionResetError:
        print(f"{client_name} disconnected.")
    finally:
        conn.close()
        if client_name in connections:
            del connections[client_name]

In [4]:
def start_server(port, client_name):
    """
    Starts a server socket listening on a specified port for a specific client.
    """
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('127.0.0.1', port))
    server_socket.listen(5)
    server_sockets.append(server_socket)  # Track the server socket
    print(f"Server listening for {client_name} on port {port}...")
    try:
        while True:
            conn, addr = server_socket.accept()
            threading.Thread(target=handle_client, args=(conn, addr, client_name)).start()
    except Exception as e:
        print(f"Server for {client_name} on port {port} encountered an error: {e}")
    finally:
        server_socket.close()

In [5]:
def send_message_to(client_name, message):
    """
    Sends a message to a connected client.
    """
    if client_name in connections:
        try:
            encodedMessage = message.encode()
            connections[client_name].send(encodedMessage)
            print(f"Sending message: {encodedMessage}")
        except BrokenPipeError:
            print(f"Failed to send message to {client_name}. Connection might be closed.")
    else:
        print(f"{client_name} is not connected.")

In [6]:
def close_all_connections():
    """
    Closes all client connections and server sockets.
    """
    print("Closing all connections...")
    for client_name, conn in list(connections.items()):
        try:
            conn.close()
            print(f"Closed connection for {client_name}")
        except Exception as e:
            print(f"Error closing connection for {client_name}: {e}")
        finally:
            del connections[client_name]

    for server_socket in server_sockets:
        try:
            server_socket.close()
            print("Server socket closed.")
        except Exception as e:
            print(f"Error closing server socket: {e}")

In [7]:
def run_servers():
    """
    Starts server threads and keeps them running in a while loop.
    """
    threading.Thread(target=start_server, args=(3000, "Java")).start()
    # threading.Thread(target=start_server, args=(4200, "Unity")).start()

    try:
        while True:
            time.sleep(1)  # Keep the main thread alive
    except KeyboardInterrupt:
        print("Shutting down servers...")
        close_all_connections()

In [ ]:
run_servers()